In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import statistics as st 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt

# EDA

In [223]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [224]:
train.count()

listing_id              20254
title                   20254
address                 20254
property_name           20254
property_type           20254
tenure                  18531
built_year              19332
num_beds                20174
num_baths               19820
size_sqft               20254
floor_level              3508
furnishing              20254
available_unit_types    18813
total_num_units         14602
property_details_url    20254
lat                     20254
lng                     20254
elevation               20254
subzone                 20141
planning_area           20141
price                   20254
dtype: int64

In [225]:
train.drop_duplicates().count()

listing_id              20254
title                   20254
address                 20254
property_name           20254
property_type           20254
tenure                  18531
built_year              19332
num_beds                20174
num_baths               19820
size_sqft               20254
floor_level              3508
furnishing              20254
available_unit_types    18813
total_num_units         14602
property_details_url    20254
lat                     20254
lng                     20254
elevation               20254
subzone                 20141
planning_area           20141
price                   20254
dtype: int64

### Remove listing_id  
#### listing_id is all unique without duplicates

In [226]:
train = train.drop(columns=['listing_id', 'property_details_url', 'elevation','property_name','address','title'])
test = test.drop(columns=['listing_id', 'property_details_url', 'elevation','property_name','address','title'])

## Adding Distances

### Add distance to commericial centres

In [227]:
ccdistance = pd.read_csv('auxiliary-data/sg-commerical-centres.csv')

In [228]:
# vectorized haversine function
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km


In [229]:
# train['cc_distance'] = haversine_np(train['lng'], train['lat'], ccdistance['lng'], ccdistance['lat'])
expected_result = pd.merge(train, ccdistance, on = 'planning_area')
expected_result.head()
train['cc_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])

# train['cc_distance'] = haversine_np(train['lng'], train['lat'], ccdistance['lng'], ccdistance['lat'])
expected_result = pd.merge(test, ccdistance, on = 'planning_area')
expected_result.head()
test['cc_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

In [230]:
ccdistancemean = train['cc_distance'].mean()
train['cc_distance']=train['cc_distance'].fillna(ccdistancemean)

ccdistancemean = test['cc_distance'].mean()
test['cc_distance']=test['cc_distance'].fillna(ccdistancemean)

### Add distance to mrt stations

In [231]:
# using long and lat
mrtdistance = pd.read_csv('auxiliary-data/sg-mrt-stations.csv')
expected_result = pd.merge(train, mrtdistance, on = 'planning_area')
train['mrt_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])


# using long and lat
mrtdistance = pd.read_csv('auxiliary-data/sg-mrt-stations.csv')
expected_result = pd.merge(test, mrtdistance, on = 'planning_area')
test['mrt_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

### Add distance to primary school 

In [232]:
# using long and lat
primaryschooldistance = pd.read_csv('auxiliary-data/sg-primary-schools.csv')
expected_result = pd.merge(train, primaryschooldistance, on = 'planning_area')
train['primary_school_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])

# using long and lat
primaryschooldistance = pd.read_csv('auxiliary-data/sg-primary-schools.csv')
expected_result = pd.merge(test, primaryschooldistance, on = 'planning_area')
test['primary_school_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

### Add distance to secondary school

In [233]:
# using long and lat
secschooldistance = pd.read_csv('auxiliary-data/sg-secondary-schools.csv')
expected_result = pd.merge(train, secschooldistance, on = 'planning_area')
train['secondary_school_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])

# using long and lat
secschooldistance = pd.read_csv('auxiliary-data/sg-secondary-schools.csv')
expected_result = pd.merge(test, secschooldistance, on = 'planning_area')
test['secondary_school_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

### Add distance to shopping malls

In [234]:
# using long and lat
shop_distance = pd.read_csv('auxiliary-data/sg-shopping-malls.csv')
expected_result = pd.merge(train, secschooldistance, on = 'planning_area')
train['shop_distance'] = haversine_np(train['lng'], train['lat'], expected_result['lng_y'], expected_result['lat_y'])

# using long and lat
shop_distance = pd.read_csv('auxiliary-data/sg-shopping-malls.csv')
expected_result = pd.merge(test, secschooldistance, on = 'planning_area')
test['shop_distance'] = haversine_np(test['lng'], test['lat'], expected_result['lng_y'], expected_result['lat_y'])

In [235]:
train.head()

,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,lat,lng,subzone,planning_area,price,cc_distance,mrt_distance,primary_school_distance,secondary_school_distance,shop_distance
0,hdb 4 rooms,NaN,1988.0,3.0,2.0,1115,NaN,unspecified,NaN,116.0,1.414399,103.837196,yishun south,yishun,514500.0,8.027727,1.689357,5.178683,2.538065,2.538065
1,hdb,99-year leasehold,1992.0,4.0,2.0,1575,NaN,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,1.372597,103.875625,serangoon north,serangoon,995400.0,2.400374,6.871207,8.310177,6.710202,6.710202
2,condo,freehold,2022.0,4.0,6.0,3070,low,partial,"studio, 3, 4, 5, 6 br",56.0,1.298773,103.895798,mountbatten,marine parade,8485000.0,6.389766,13.983476,9.485153,15.197137,15.197137
3,Condo,freehold,2023.0,3.0,2.0,958,NaN,partial,"studio, 1, 2, 3, 4 br",638.0,1.312364,103.803271,farrer court,bukit timah,2626000.0,8.802339,13.479646,3.290085,12.950660,12.950660
4,condo,99-year leasehold,2026.0,2.0,1.0,732,NaN,unspecified,"studio, 1, 2, 3, 4, 5 br",351.0,1.273959,103.843635,anson,downtown core,1764000.0,9.160635,15.981982,13.818954,17.977600,17.977600


In [236]:
train['cc_distance'].describe()

count    20254.000000
mean        30.538219
std        332.696968
min          0.089512
25%          7.140757
50%         12.737956
75%         21.287429
max      15529.817405
Name: cc_distance, dtype: float64

## Property Type cleaning

In [237]:
train.property_type.unique()

array(['hdb 4 rooms', 'hdb', 'condo', 'Condo', 'bungalow', 'Hdb',
       'terraced house', 'Hdb Executive', 'apartment',
       'Semi-Detached House', 'Apartment', 'Hdb 4 Rooms',
       'semi-detached house', 'hdb 3 rooms', 'executive condo',
       'corner terrace', 'hdb executive', 'Hdb 3 Rooms', 'Hdb 5 Rooms',
       'hdb 5 rooms', 'landed', 'hdb 2 rooms', 'Executive Condo',
       'Bungalow', 'Corner Terrace', 'Terraced House', 'cluster house',
       'Cluster House', 'Land Only', 'townhouse', 'Hdb 2 Rooms',
       'conservation house', 'land only', 'walk-up', 'Townhouse',
       'Conservation House', 'good class bungalow', 'Landed', 'shophouse'],
      dtype=object)

In [238]:
#Change all to uppercase
train['property_type']=train['property_type'].str.upper()

#Change all to uppercase
test['property_type']=test['property_type'].str.upper()

In [239]:
train.property_type.unique()

array(['HDB 4 ROOMS', 'HDB', 'CONDO', 'BUNGALOW', 'TERRACED HOUSE',
       'HDB EXECUTIVE', 'APARTMENT', 'SEMI-DETACHED HOUSE', 'HDB 3 ROOMS',
       'EXECUTIVE CONDO', 'CORNER TERRACE', 'HDB 5 ROOMS', 'LANDED',
       'HDB 2 ROOMS', 'CLUSTER HOUSE', 'LAND ONLY', 'TOWNHOUSE',
       'CONSERVATION HOUSE', 'WALK-UP', 'GOOD CLASS BUNGALOW',
       'SHOPHOUSE'], dtype=object)

In [240]:
train[train['property_type'] == 'CORNER TERRACE']

,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,lat,lng,subzone,planning_area,price,cc_distance,mrt_distance,primary_school_distance,secondary_school_distance,shop_distance
84,CORNER TERRACE,999-year leasehold,NaN,5.0,5.0,6437,NaN,unspecified,"4, 5, 6 br",19.0,1.372693,103.769918,dairy farm,bukit panjang,9816300.0,11.620482,9.594432,2.712021,10.531181,10.531181
339,CORNER TERRACE,NaN,NaN,5.0,7.0,5500,NaN,unspecified,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10 br",NaN,1.360245,103.865111,serangoon garden,serangoon,7140000.0,1.278840,8.385522,11.452909,7.147379,7.147379
368,CORNER TERRACE,freehold,NaN,4.0,4.0,3200,NaN,unspecified,"1, 3, 4 br",NaN,1.313186,103.899693,geylang east,geylang,5124000.0,5.211469,13.061795,9.139648,15.697107,15.697107
535,CORNER TERRACE,99-year leasehold,2023.0,4.0,5.0,3466,NaN,unspecified,"studio, 1, 2, 3, 4, 5 br",1468.0,1.317138,103.759701,faber,clementi,4756500.0,13.070656,13.797390,3.539115,15.500131,15.500131
579,CORNER TERRACE,99-year leasehold,1999.0,7.0,6.0,3300,NaN,unspecified,"5, 6, 7, 8 br",73.0,1.347540,103.881737,tai seng,hougang,3465000.0,1.127084,10.478072,19.339842,9.164133,9.164133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19907,CORNER TERRACE,NaN,NaN,5.0,6.0,2800,NaN,unspecified,5 br,NaN,1.312759,103.925318,frankel,bedok,8400000.0,30.538219,7.512741,0.786513,19.578680,19.578680
19947,CORNER TERRACE,999-year leasehold,NaN,5.0,2.0,2500,NaN,partial,"3, 4, 5, 6, 7, 8, 10 br",NaN,1.387049,103.872999,seletar hills,serangoon,4410000.0,30.538219,12.474262,9.581267,14.185692,14.185692
20093,CORNER TERRACE,freehold,2005.0,5.0,6.0,2421,NaN,unspecified,5 br,67.0,1.364266,103.865114,serangoon garden,serangoon,3045000.0,30.538219,7.826439,12.051209,13.394508,13.394508
20110,CORNER TERRACE,freehold,1987.0,6.0,6.0,2800,NaN,unspecified,"5, 6 br",73.0,1.382537,103.773849,bangkit,bukit panjang,5649000.0,30.538219,13.245863,11.251458,3.279932,3.279932


In [241]:
# TO DO: MAY NEED TO CHECK EACH VALUE INDIVIDUALLY TO SEE IF THERE ARE ANY WITH ONLY 1 

In [242]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train['property_type'])
train['property_type']=le.transform(train['property_type'])

test['property_type']=le.transform(test['property_type'])

In [243]:
train.head()

,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,lat,lng,subzone,planning_area,price,cc_distance,mrt_distance,primary_school_distance,secondary_school_distance,shop_distance
0,11,NaN,1988.0,3.0,2.0,1115,NaN,unspecified,NaN,116.0,1.414399,103.837196,yishun south,yishun,514500.0,8.027727,1.689357,5.178683,2.538065,2.538065
1,8,99-year leasehold,1992.0,4.0,2.0,1575,NaN,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,1.372597,103.875625,serangoon north,serangoon,995400.0,2.400374,6.871207,8.310177,6.710202,6.710202
2,3,freehold,2022.0,4.0,6.0,3070,low,partial,"studio, 3, 4, 5, 6 br",56.0,1.298773,103.895798,mountbatten,marine parade,8485000.0,6.389766,13.983476,9.485153,15.197137,15.197137
3,3,freehold,2023.0,3.0,2.0,958,NaN,partial,"studio, 1, 2, 3, 4 br",638.0,1.312364,103.803271,farrer court,bukit timah,2626000.0,8.802339,13.479646,3.290085,12.950660,12.950660
4,3,99-year leasehold,2026.0,2.0,1.0,732,NaN,unspecified,"studio, 1, 2, 3, 4, 5 br",351.0,1.273959,103.843635,anson,downtown core,1764000.0,9.160635,15.981982,13.818954,17.977600,17.977600


## Tenure 

In [244]:
train.tenure.unique()

array([nan, '99-year leasehold', 'freehold', '999-year leasehold',
       '110-year leasehold', '946-year leasehold', '103-year leasehold',
       '956-year leasehold', '929-year leasehold', '102-year leasehold',
       '100-year leasehold', '947-year leasehold'], dtype=object)

In [245]:
train['tenure'] = train['tenure'].replace(
    [
    '99-year leasehold',
    '110-year leasehold',
    '103-year leasehold',
    '102-year leasehold',
    '100-year leasehold',
    ],
    '99-year leasehold'
)

train['tenure'] = train['tenure'].replace(
    [
    '946-year leasehold',
    '999-year leasehold',
    '956-year leasehold',
    '929-year leasehold',
    '947-year leasehold',
    ],
    '999-year leasehold'
)



test['tenure'] = test['tenure'].replace(
    [
    '99-year leasehold',
    '110-year leasehold',
    '103-year leasehold',
    '102-year leasehold',
    '100-year leasehold',
    ],
    '99-year leasehold'
)

test['tenure'] = test['tenure'].replace(
    [
    '946-year leasehold',
    '999-year leasehold',
    '956-year leasehold',
    '929-year leasehold',
    '947-year leasehold',
    ],
    '999-year leasehold'
)

In [246]:
train.tenure.unique()

array([nan, '99-year leasehold', 'freehold', '999-year leasehold'],
      dtype=object)

In [247]:
test['tenure'] = test['tenure'].fillna('99-year leasehold')

In [248]:
## Remove NA 
train = train.dropna(subset=['tenure'])

## Remove NA 
test['tenure'] = test['tenure'].fillna('99-year leasehold')

In [249]:
from sklearn import preprocessing
tenurepp = preprocessing.LabelEncoder()
tenurepp.fit(train['tenure'])
train['tenure']=tenurepp.transform(train['tenure'])

test['tenure']=tenurepp.transform(test['tenure'])

### Funishing

In [250]:
train.furnishing.unique()

array(['unspecified', 'partial', 'unfurnished', 'fully', 'na'],
      dtype=object)

In [251]:
#Assume not specified and na as unfurnished 
train['furnishing'] = train['furnishing'].replace(
    [
    'na',
    'unspecified',
    ],
    'unfurnished'
)

#Assume not specified and na as unfurnished 
test['furnishing'] = test['furnishing'].replace(
    [
    'na',
    'unspecified',
    ],
    'unfurnished'
)

In [252]:
train['furnishing'].unique()

array(['unfurnished', 'partial', 'fully'], dtype=object)

In [253]:
from sklearn import preprocessing
furnishing = preprocessing.LabelEncoder()
furnishing.fit(train['furnishing'])
train['furnishing']=furnishing.transform(train['furnishing'])

test['furnishing']=furnishing.transform(test['furnishing'])

## floor_level 

In [254]:
train['floor_level'].count()

3238

In [255]:
# Too few data for floor level, we should just take out the column
train = train.drop(columns='floor_level')

# Too few data for floor level, we should just take out the column
test = test.drop(columns='floor_level')

## total_num_units

In [256]:
train.head()

,property_type,tenure,built_year,num_beds,num_baths,size_sqft,furnishing,available_unit_types,total_num_units,lat,lng,subzone,planning_area,price,cc_distance,mrt_distance,primary_school_distance,secondary_school_distance,shop_distance
1,8,0,1992.0,4.0,2.0,1575,2,"1, 2, 3, 4, 5, 6 br",NaN,1.372597,103.875625,serangoon north,serangoon,995400.0,2.400374,6.871207,8.310177,6.710202,6.710202
2,3,2,2022.0,4.0,6.0,3070,1,"studio, 3, 4, 5, 6 br",56.0,1.298773,103.895798,mountbatten,marine parade,8485000.0,6.389766,13.983476,9.485153,15.197137,15.197137
3,3,2,2023.0,3.0,2.0,958,1,"studio, 1, 2, 3, 4 br",638.0,1.312364,103.803271,farrer court,bukit timah,2626000.0,8.802339,13.479646,3.290085,12.950660,12.950660
4,3,0,2026.0,2.0,1.0,732,2,"studio, 1, 2, 3, 4, 5 br",351.0,1.273959,103.843635,anson,downtown core,1764000.0,9.160635,15.981982,13.818954,17.977600,17.977600
5,3,2,2024.0,2.0,2.0,689,2,"studio, 1, 2, 3, 4, 5 br",633.0,1.339338,103.763893,bukit batok south,bukit batok,1702000.0,12.115573,8.822150,9.964453,11.837034,11.837034


In [257]:
train['total_num_units'].count()

13161

In [258]:
train[train['total_num_units'].isna()]

,property_type,tenure,built_year,num_beds,num_baths,size_sqft,furnishing,available_unit_types,total_num_units,lat,lng,subzone,planning_area,price,cc_distance,mrt_distance,primary_school_distance,secondary_school_distance,shop_distance
1,8,0,1992.0,4.0,2.0,1575,2,"1, 2, 3, 4, 5, 6 br",NaN,1.372597,103.875625,serangoon north,serangoon,995400.0,2.400374,6.871207,8.310177,6.710202,6.710202
7,1,1,NaN,5.0,6.0,11500,2,"2, 3, 4, 5, 6, 7 br",NaN,1.342340,103.784489,swiss club,bukit timah,15645000.0,9.805071,9.927822,17.317227,11.659117,11.659117
16,8,0,1981.0,3.0,2.0,980,2,3 br,NaN,1.328739,103.844534,balestier,novena,554400.0,3.971491,9.933887,8.685316,12.037872,12.037872
19,8,0,1980.0,1.0,1.0,470,2,"1, 2, 3, 4, 5 br",NaN,1.373794,103.849014,cheng san,ang mo kio,309800.0,3.600547,5.161016,6.396035,5.296377,5.296377
22,11,0,1995.0,4.0,2.0,1625,2,"studio, 1, 2, 3, 4, 5, 6 br",NaN,1.294376,103.804483,mei chin,queenstown,1134000.0,9.833951,14.030994,5.270583,15.808970,15.808970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20237,8,0,1984.0,3.0,2.0,1453,2,"1, 2, 3, 4, 5, 6 br",NaN,1.327859,103.853387,balestier,novena,924000.0,30.538219,4.885928,18.385328,11.415664,11.415664
20240,10,0,1978.0,2.0,2.0,720,2,"2, 3, 4, 5 br",NaN,1.320406,103.884263,aljunied,geylang,399000.0,30.538219,5.450232,1.106325,15.656486,15.656486
20243,9,0,1976.0,2.0,1.0,699,2,"studio, 2, 3 br",NaN,1.347253,103.713914,boon lay place,jurong west,367500.0,30.538219,17.311327,1.088211,3.856739,3.856739
20244,10,0,1983.0,2.0,2.0,797,1,"studio, 2, 3, 4 br",NaN,1.424190,103.844932,yishun east,yishun,386400.0,30.538219,13.981805,12.736755,12.894390,12.894390


In [259]:
train['total_num_units'].describe()

count    13161.000000
mean       405.463187
std        352.977961
min          5.000000
25%        119.000000
50%        340.000000
75%        605.000000
max       2612.000000
Name: total_num_units, dtype: float64

In [260]:
meantotalunit = train['total_num_units'].mean()
train['total_num_units'].fillna(meantotalunit, inplace=True)

meantotalunit = test['total_num_units'].mean()
test['total_num_units'].fillna(meantotalunit, inplace=True)

## build year

In [261]:
avgbuildyear = round(train['built_year'].mean(),0)
train['built_year'] = train['built_year'].fillna(avgbuildyear)

avgbuildyear = round(test['built_year'].mean(),0)
test['built_year'] = test['built_year'].fillna(avgbuildyear)

## num_baths

In [262]:
train.head()

,property_type,tenure,built_year,num_beds,num_baths,size_sqft,furnishing,available_unit_types,total_num_units,lat,lng,subzone,planning_area,price,cc_distance,mrt_distance,primary_school_distance,secondary_school_distance,shop_distance
1,8,0,1992.0,4.0,2.0,1575,2,"1, 2, 3, 4, 5, 6 br",405.463187,1.372597,103.875625,serangoon north,serangoon,995400.0,2.400374,6.871207,8.310177,6.710202,6.710202
2,3,2,2022.0,4.0,6.0,3070,1,"studio, 3, 4, 5, 6 br",56.000000,1.298773,103.895798,mountbatten,marine parade,8485000.0,6.389766,13.983476,9.485153,15.197137,15.197137
3,3,2,2023.0,3.0,2.0,958,1,"studio, 1, 2, 3, 4 br",638.000000,1.312364,103.803271,farrer court,bukit timah,2626000.0,8.802339,13.479646,3.290085,12.950660,12.950660
4,3,0,2026.0,2.0,1.0,732,2,"studio, 1, 2, 3, 4, 5 br",351.000000,1.273959,103.843635,anson,downtown core,1764000.0,9.160635,15.981982,13.818954,17.977600,17.977600
5,3,2,2024.0,2.0,2.0,689,2,"studio, 1, 2, 3, 4, 5 br",633.000000,1.339338,103.763893,bukit batok south,bukit batok,1702000.0,12.115573,8.822150,9.964453,11.837034,11.837034


In [269]:
a = train[train['num_baths'].isna()]
a[a['num_beds'].isna()]

,property_type,tenure,built_year,num_beds,num_baths,size_sqft,furnishing,available_unit_types,total_num_units,lat,lng,subzone,planning_area,price,cc_distance,mrt_distance,primary_school_distance,secondary_school_distance,shop_distance
3713,1,2,1990.0,NaN,NaN,9594,2,"studio, 1, 4, 5, 6, 7 br",405.463187,1.319714,103.829907,malcolm,novena,16275000.0,5.668353,5.872226,6.905858,11.134265,11.134265
4691,3,0,2023.0,NaN,NaN,431,2,"studio, 1, 2, 3, 4, 5, 8 br",774.000000,1.283016,103.839887,pearl's hill,outram,1222200.0,5.825357,8.355056,11.798265,15.372730,15.372730
11500,8,0,1985.0,NaN,NaN,786,2,"studio, 2, 3, 4 br",405.463187,1.349904,103.756987,bukit batok east,bukit batok,409500.0,8.707527,4.914059,6.279880,18.454836,18.454836


In [138]:
avgbath = round(train['num_baths'].mean(),1)
train['num_baths'] = train['num_baths'].fillna(avgbath)

avgbath = round(test['num_baths'].mean(),1)
test['num_baths'] = test['num_baths'].fillna(avgbath)

## num_bedds

In [139]:
avgbed = round(train['num_beds'].mean(),1)
train['num_beds'] = train['num_beds'].fillna(avgbath)

avgbed = round(test['num_beds'].mean(),1)
test['num_beds'] = test['num_beds'].fillna(avgbath)

## subzone&planning zone are very important to price of realestate. So we can drop na listings without it.

## num_avaliable_units

In [140]:
train.head()

,property_type,tenure,built_year,num_beds,num_baths,size_sqft,furnishing,available_unit_types,total_num_units,lat,lng,subzone,planning_area,price,cc_distance,mrt_distance,primary_school_distance,secondary_school_distance,shop_distance
1,8,0,1992.0,4.0,2.0,1575,2,"1, 2, 3, 4, 5, 6 br",405.463187,1.372597,103.875625,serangoon north,serangoon,995400.0,2.400374,6.871207,8.310177,6.710202,6.710202
2,3,2,2022.0,4.0,6.0,3070,1,"studio, 3, 4, 5, 6 br",56.000000,1.298773,103.895798,mountbatten,marine parade,8485000.0,6.389766,13.983476,9.485153,15.197137,15.197137
3,3,2,2023.0,3.0,2.0,958,1,"studio, 1, 2, 3, 4 br",638.000000,1.312364,103.803271,farrer court,bukit timah,2626000.0,8.802339,13.479646,3.290085,12.950660,12.950660
4,3,0,2026.0,2.0,1.0,732,2,"studio, 1, 2, 3, 4, 5 br",351.000000,1.273959,103.843635,anson,downtown core,1764000.0,9.160635,15.981982,13.818954,17.977600,17.977600
5,3,2,2024.0,2.0,2.0,689,2,"studio, 1, 2, 3, 4, 5 br",633.000000,1.339338,103.763893,bukit batok south,bukit batok,1702000.0,12.115573,8.822150,9.964453,11.837034,11.837034


In [104]:
numavaunittypes = train.available_unit_types.str.count(',') + 1n
train['available_unit_types'] = numavaunittypes

In [105]:
avg_ava_units = train['available_unit_types'].mean()
train['available_unit_types'] = train['available_unit_types'].fillna(avg_ava_units)

In [106]:
numavaunittypes = test.available_unit_types.str.count(',') +1
test['available_unit_types'] = numavaunittypes

avg_ava_units = test['available_unit_types'].mean()
test['available_unit_types'] = test['available_unit_types'].fillna(avg_ava_units)

## Subzone & planning zone

In [107]:
train['subzone'].unique()

array(['serangoon north', 'mountbatten', 'farrer court', 'anson',
       'bukit batok south', 'kampong java', 'swiss club', 'nassim',
       'leedon park', 'bedok south', 'kaki bukit', 'upper thomson',
       'balestier', 'lavender', 'cecil', 'cheng san', 'dover',
       'flora drive', 'mei chin', 'bras basah', 'cairnhill', nan,
       'moulmein', 'frankel', 'ridout', 'yishun east', 'dhoby ghaut',
       'upper paya lebar', 'boulevard', 'holland drive', 'fajar',
       'gombak', 'bedok reservoir', 'sembawang central', 'seletar hills',
       'aljunied', 'bedok north', 'everton park', 'pasir ris drive',
       'bugis', 'anchorvale', 'pasir ris west', 'chatsworth', 'joo seng',
       'sembawang springs', 'newton circus', 'toh tuck', 'chinatown',
       'tanjong rhu', 'north coast', 'crawford', 'dairy farm',
       'bendemeer', 'lorong ah soo', 'robertson quay', 'hong kah north',
       'fernvale', 'woodgrove', 'geylang east', 'telok blangah drive',
       'lower seletar', 'kampong ubi', 

In [35]:
test.count()

property_type                7000
tenure                       7000
built_year                   7000
num_beds                     7000
num_baths                    7000
size_sqft                    7000
furnishing                   7000
available_unit_types         7000
total_num_units              7000
lat                          7000
lng                          7000
subzone                      6967
planning_area                6967
cc_distance                  7000
mrt_distance                 7000
primary_school_distance      7000
secondary_school_distance    7000
shop_distance                7000
dtype: int64

In [36]:
train['planning_area'].unique()

array(['serangoon', 'marine parade', 'bukit timah', 'downtown core',
       'bukit batok', 'kallang', 'tanglin', 'bedok', 'bishan', 'novena',
       'ang mo kio', 'queenstown', 'pasir ris', 'museum', 'newton', nan,
       'yishun', 'orchard', 'bukit panjang', 'sembawang', 'geylang',
       'bukit merah', 'sengkang', 'toa payoh', 'clementi', 'outram',
       'woodlands', 'hougang', 'singapore river', 'punggol',
       'river valley', 'jurong west', 'southern islands', 'tampines',
       'rochor', 'jurong east', 'choa chu kang',
       'central water catchment', 'tengah', 'lim chu kang', 'changi',
       'seletar', 'paya lebar', 'mandai'], dtype=object)

In [37]:
pa = train['planning_area'] + test['planning_area']
pa = pa.dropna().drop_duplicates().unique()
pa = pd.DataFrame(pa)
pa.count()

0    1255
dtype: int64

In [38]:
a = train['planning_area'].drop_duplicates()
b = test['planning_area'].drop_duplicates()
c = pd.concat([a, b]).dropna()
c = c.drop_duplicates()


In [39]:
from sklearn import preprocessing
planningzone = preprocessing.LabelEncoder()
planningzone.fit(c)

train = train.dropna()
train['planning_area']=planningzone.transform(train['planning_area'])

In [40]:
mostfreq = test['planning_area'].mode()
mostfreq
test['planning_area'] = test['planning_area'].fillna('bukit timah')
test['planning_area']=planningzone.transform(test['planning_area'])

## CAN CONSIDER NOT DROPPING SUBZONE

In [41]:
train = train.drop(columns='subzone')
test = test.drop(columns='subzone')

## Data MINING

In [42]:
'''Main'''
import numpy as np
import pandas as pd
from time import time

'''Data Prep'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report

# data visualisation
import matplotlib.pyplot as plt

from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier as xgb

In [43]:
dataX = train.copy()
dataY = dataX.pop('price')

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.2, random_state=88)

In [44]:
# important sample code
# parameter tuning for random forest
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

params = {'max_features':('sqrt', 'log2'), 'n_estimators':range(10, 100, 5)} # from 10 to 100 with 5 increment

clf = RandomForestClassifier(
    n_jobs=-1, # for using multiple cpu/core to speed up calculation
    random_state=2022, # random seed
    verbose=0, # print out details or not    
)

clf2 = GridSearchCV(clf, param_grid=params, scoring='accuracy', cv=5)
clf2.fit(X_train, y_train)

# see all output variables in cv results
sorted(clf2.cv_results_.keys())

KeyboardInterrupt: 

In [ ]:
clf2.best_score_

In [ ]:
# get the best model
clf2.best_estimator_

In [ ]:
print("Accuracy on training set: {:.3f}".format(clf2.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(clf2.score(X_test, y_test)))